<table>
    <tr>
      <td>Introducción a
      </td>
      <td>
      <img src="https://media.licdn.com/dms/image/D5612AQF7GSp3l4pztQ/article-cover_image-shrink_720_1280/0/1686548640655?e=1715817600&v=beta&t=WQzv1EMkEEwZ0QZ0PF1anRKIHCl5BBH_YPZHdDQsWPM"  width=150/>
      </td>
     </tr>
</table>
Rafa Caballero


## Evaluación

Con la validación cruzada hemos visto como:

- Evaluar de forma fiable lo bueno que es un modelo (con `cross_val_score`)

- Obtener los mejores valores para los hiperparámetros (con ayuda de `GridSearchCV` y `RandomizedSearchCV`)

- Sin embargo quedan preguntas por resolver como  ¿Podemos mejorar los resultados con más datos? ¿tenemos overfitting?

Las curvas de aprendizaje son un buen mecanismo para ver esto. La idea principal es:

- El error sobre test es siempre mayor que el que se tiene sobre train, por eso train es una inferior del error

- Tener en cuenta que cuando no hablamos de error sino de score es al reves, el train es una cota superior del error

Veamos un ejemplo

In [ ]:
!pip install ipython-autotime
%load_ext autotime


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.naive_bayes import BernoulliNB
from sklearn.feature_extraction.text import CountVectorizer
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt

import spacy

#### Carga y preprocesado
path = "https://raw.githubusercontent.com/amankharwal/SMS-Spam-Detection/master/spam.csv"
#path = "c:/hlocal/tdm/movimiento.csv"
df = pd.read_csv(path,encoding="latin1").drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1)
df["label"]=0
df.loc[df["class"]=="spam","label"] =1

nlp = spacy.blank('en')
stopwords = nlp.Defaults.stop_words

l = [" ".join([token.text.upper() for token in nlp(doc) if not token.is_stop and  token.is_alpha]) for doc in df.message]

df["limpio"] = l
frases =df["limpio"].values
y = df["label"].values
# creating count vectorizer object
countV = CountVectorizer()
#tranforming values
X = countV.fit_transform(frases)

Utilizamos `cross_validate` en lugar de `cross_val_score`porque con el parámetro `train_score=True` nos devuelve los errores sobre el entrenamiento además de sobre el test

In [ ]:
from sklearn.metrics import make_scorer
from sklearn.model_selection import RepeatedStratifiedKFold,cross_val_score,cross_validate
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.metrics import cohen_kappa_score

steps = [('BernoulliNB', BernoulliNB())]
scorer = make_scorer(cohen_kappa_score)
pipeline = Pipeline(steps=steps)
cv = RepeatedStratifiedKFold(n_splits=20, n_repeats=5)
cv_results = cross_validate(pipeline, X, y, scoring=scorer, cv=cv,return_train_score=True)
cv_results.keys()

In [ ]:
cv_results["test_score"]

In [ ]:
cv_results["train_score"]

Como vemos en general los valores sobre train serán mejores que sobre test

In [ ]:
cv_results["test_score"].mean(), cv_results["train_score"].mean()

Sin embargo esta información todavía no es útil, obtendremos una visión más clara si tenemos estos datos para distintas proporciones de train y test. Hagámoslo primero manualmente:

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.naive_bayes import MultinomialNB
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
import math
from tqdm import tqdm
from sklearn.metrics import make_scorer,f1_score, accuracy_score, recall_score,cohen_kappa_score
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline



veces = 50
gap=5
total=100
resultados = []
for i in tqdm(range(gap,total-gap)):
    test = 1-i/total
    #print("test",test)
    resultados_train = []
    resultados_test = []
    for v in  range(veces): #tqdm(range(veces), desc =str(i)+"-"+str(round(test,3))):
        # 2
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= test)
        #print(len(X_train),len(X_test))
        # 3
        metodo = MultinomialNB()
        modelo = metodo.fit(X_train,y_train)

        # 4
        y_pred_test = modelo.predict(X_test)
        kappa_test = cohen_kappa_score(y_test,y_pred_test)
        resultados_test.append(kappa_test)

        y_pred_train = modelo.predict(X_train)
        kappa_train =cohen_kappa_score(y_train,y_pred_train)
        resultados_train.append(kappa_train)
        #print(mae_train,mae_test)

    #### acumulamos resultados
    valores = (test, np.array(resultados_train).mean(), np.array(resultados_test).mean())
    #print(valores)
    resultados.append(valores)


In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))
y_train = [b for a,b,c in resultados ]
y_test = [c for a,b,c in resultados ]
x = [1-a for a,b,c in resultados ]

ax.scatter(x,y_train,color="blue")
ax.plot(x,y_train,label="train",color="blue")
ax.scatter(x,y_test,color="orange")
ax.plot(x,y_test,label="test",color="orange")
plt.legend()
plt.title("Curva de aprendizaje")
plt.show()

In [ ]:
min(y_train),max(y_test)

Vemos que las curvas son prácticamente estables, hay poca diferencia. El test
podría mejorar con más datos pero nunca por encima de 0.94 (posiblemente menos alrededor de 0.92). Es un sistema que aprende muy rápido y se estabiliza con pocos datos, luego la mejora es muy lenta

En lugar de hacerlo a mano podemos hacerlo con el método `learning_curve`

In [ ]:
from sklearn.model_selection import learning_curve
veces = 50
gap=5
total=100
entrena = [i/total for i in range(gap,total-gap)]
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5)
metodo = MultinomialNB()
train_size_abs, train_scores, test_scores = learning_curve( metodo, X, y,train_sizes=entrena,
                                                           cv=cv,verbose=2,n_jobs=-1,scoring=scorer,shuffle=True)

Ahora representamos ambos valores, los de entrenamiento y los de test

In [ ]:
len(train_scores), len(train_scores[0])

In [ ]:
trainS = [s.mean() for s in train_scores]
testS = [s.mean() for s in test_scores]
coordX = list(range(len(trainS)))
fig, ax = plt.subplots(figsize=(15, 5))
ax.scatter(x,y_train,color="blue")
ax.plot(x,y_train,label="train",color="blue")
ax.scatter(x,y_test,color="orange")
ax.plot(x,y_test,label="test",color="orange")
plt.legend()
plt.title("Curva de aprendizaje")
plt.show()

Hay incluso un método que calcula los valores y muestra la gráfica, todo junto

In [ ]:
from sklearn.model_selection import LearningCurveDisplay


fig, ax = plt.subplots(figsize=(15, 5))
common_params = {
    "X": X,
    "y": y,
    "train_sizes": np.linspace(0.05, 0.95, 100),
    "cv": cv,
    "score_type": "both",
    "n_jobs": -1,
    "line_kw": {"marker": "o"},
    "std_display_style": "fill_between",
    "score_name": "kappa",
    "scoring":scorer
}

LearningCurveDisplay.from_estimator(metodo, **common_params, ax=ax)
plt.show()

Veamos otro ejemplo:

In [ ]:
url = "https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/usa2020/twords.csv"
import pandas as pd

from sklearn.naive_bayes import MultinomialNB

df = pd.read_csv(url)
df = df.drop(columns=["id"])
label = "label"
XColumns = [c for c in df.columns if c!=label]
yColumn = label
X = df[XColumns]
y = df[yColumn]


In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=1)
common_params = {
    "X": X,
    "y": y,
    "train_sizes": np.linspace(0.1, 0.95, 100),
    "cv": cv,
    "score_type": "both",
    "n_jobs": -1,
    "line_kw": {"marker": "o"},
    "std_display_style": "fill_between",
    "score_name": "kappa",
    "scoring":scorer
}

LearningCurveDisplay.from_estimator(MultinomialNB(), **common_params, ax=ax)
plt.show()

¿Qué consecuencias se pueden sacar? ¿Vendría bien tener más datos para entrenar?

Aún un tercer ejemplo

In [ ]:
file = "https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/bus.csv"
import pandas as pd
from sklearn.linear_model import LogisticRegression

df = pd.read_csv(file)
df["label"] = 1
df.loc[df.I8<580, "label"] = 0
df = df.drop(columns=["I8"])
yColumn = "label"
XColumns = [c for c in df.columns if c!=yColumn]
X = df[XColumns]
y = df[yColumn]


In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))
cv = RepeatedStratifiedKFold(n_splits=20, n_repeats=5)
common_params = {
    "X": X,
    "y": y,
    "train_sizes": np.linspace(0.1, 1.0, 100),
    "cv": cv,
    "score_type": "both",
    "n_jobs": -1,
    "line_kw": {"marker": "o"},
    "std_display_style": "fill_between",
    "score_name": "kappa",
    "scoring":scorer
}

LearningCurveDisplay.from_estimator(LogisticRegression(), **common_params, ax=ax)
plt.show()

¿En este caso interesa tener más datos de entrenamiento?

Es interesante ver cómo cambia la curva al cambiar el método

In [ ]:
from sklearn.naive_bayes import GaussianNB
fig, ax = plt.subplots(figsize=(15, 5))
LearningCurveDisplay.from_estimator(GaussianNB(), **common_params, ax=ax)
plt.show()

En el caso del Titanic no podemos lograr más datos de entrenamiento, pero aun así podemos ver la curva y qué sucede:

In [ ]:
url = "https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/titanicyesno.csv"
import pandas as pd
df = pd.read_csv(url)
columnas = ["survived", "pclass", "sex", "age", "sibsp", "parch", "fare","embarked"]
df2 = df[columnas]
df2 = df2.dropna()
df2["survived"] = df2.survived.replace(('yes', 'no'), (1, 0))
df2["sex"] = df2.sex.replace(('female', 'male'), (1, 0))
df = pd.get_dummies(df2, columns=["embarked"])

yColumn = "survived"
XColumns = [c for c in df.columns if c!=yColumn]
X= df[XColumns]
y=df[yColumn]

Probar con regresión logística y con GaussianNB